In [1]:
import json

data = []
with open('Omni-Math.jsonl', 'r') as f:
    for l in f:
        data.append(json.loads(l))

In [2]:
data

[{'domain': ['Mathematics -> Algebra -> Other'],
  'difficulty': 8.0,
  'problem': 'Let $ n(\\ge2) $ be a positive integer. Find the minimum $ m $, so that there exists $x_{ij}(1\\le i ,j\\le n)$ satisfying:\n(1)For every $1\\le i ,j\\le n, x_{ij}=max\\{x_{i1},x_{i2},...,x_{ij}\\} $ or $ x_{ij}=max\\{x_{1j},x_{2j},...,x_{ij}\\}.$\n(2)For every $1\\le i \\le n$, there are at most $m$ indices $k$ with $x_{ik}=max\\{x_{i1},x_{i2},...,x_{ik}\\}.$\n(3)For every $1\\le j \\le n$, there are at most $m$ indices $k$ with $x_{kj}=max\\{x_{1j},x_{2j},...,x_{kj}\\}.$',
  'solution': '\nLet \\( n (\\geq 2) \\) be a positive integer. We aim to find the minimum \\( m \\) such that there exists \\( x_{ij} \\) (for \\( 1 \\leq i, j \\leq n \\)) satisfying the following conditions:\n1. For every \\( 1 \\leq i, j \\leq n \\), \\( x_{ij} = \\max \\{ x_{i1}, x_{i2}, \\ldots, x_{ij} \\} \\) or \\( x_{ij} = \\max \\{ x_{1j}, x_{2j}, \\ldots, x_{ij} \\} \\).\n2. For every \\( 1 \\leq i \\leq n \\), there are 

In [3]:
import pandas as pd
df = pd.DataFrame(data)
df

,domain,difficulty,problem,solution,answer,source
0,[Mathematics -> Algebra -> Other],8.0,Let $ n(\ge2) $ be a positive integer. Find th...,\nLet \( n (\geq 2) \) be a positive integer. ...,1 + \left\lceil \frac{n}{2} \right\rceil,china_team_selection_test
1,[Mathematics -> Geometry -> Plane Geometry -> ...,7.0,"In an acute scalene triangle $ABC$, points $D,...","\nIn an acute scalene triangle \(ABC\), points...",1,usa_team_selection_test
2,[Mathematics -> Discrete Mathematics -> Graph ...,7.0,A tournament is a directed graph for which eve...,\nA tournament is a directed graph for which e...,\lceil \log_2 n \rceil,usa_team_selection_test
3,[Mathematics -> Algebra -> Algebra -> Polynomi...,8.5,"Does there exist positive reals $a_0, a_1,\ldo...",\n\nTo determine whether there exist positive ...,\text{Yes},china_national_olympiad
4,[Mathematics -> Algebra -> Abstract Algebra ->...,7.0,Let $p$ be a prime. We arrange the numbers in ...,\nLet \( p \) be a prime. We arrange the numbe...,2(p!)^2,china_national_olympiad
...,...,...,...,...,...,...
4423,[Mathematics -> Algebra -> Abstract Algebra ->...,8.0,Suppose that $k$ is a positive integer. A bije...,\nTo determine whether for every positive inte...,\text{Yes},balkan_mo_shortlist
4424,[Mathematics -> Algebra -> Algebra -> Algebrai...,5.0,Let $a_0$ be a positive integer and $a_n=5a_{n...,\nGiven the recurrence relation \( a_n = 5a_{n...,\text{No},baltic_way
4425,[Mathematics -> Algebra -> Intermediate Algebr...,6.0,Consider two positive integers $a$ and $b$ suc...,\nGiven two positive integers \( a \) and \( b...,$ \text { No } $,ToT
4426,[Mathematics -> Discrete Mathematics -> Combin...,6.5,Kolya and Dima play a game on an $8\times 8$ b...,\nTo analyze the problem of Kolya and Dima’s g...,"\text{Kolya wins if Dima starts, and Dima wins...",problems_from_the_kvant_magazine


In [8]:
df['label'] = ""
df['domain_hash'] = df.apply(lambda x: "".join(x['domain']), axis=1)

In [12]:
def reorder_dataframe(df, column_name, value_to_extract):
    """
    Reorders a pandas DataFrame based on the unique values in a specified column,
    extracting a certain number of rows for each unique value in an alternating fashion.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column to use for reordering.
        value_to_extract (int): The number of rows to extract for each unique value
            from the specified column.

    Returns:
        pd.DataFrame: A new DataFrame with the rows reordered as specified.  Returns
                      an empty DataFrame if the input DataFrame is empty,
                      if the column doesn't exist, or if the value_to_extract is invalid.
    """
    # Handle edge cases: empty DataFrame, invalid column name, invalid value_to_extract
    if df.empty:
        print("Input DataFrame is empty. Returning an empty DataFrame.")
        return pd.DataFrame()
    if column_name not in df.columns:
        print(f"Column '{column_name}' not found in DataFrame. Returning an empty DataFrame.")
        return pd.DataFrame()
    if not isinstance(value_to_extract, int) or value_to_extract <= 0:
        print("value_to_extract must be a positive integer. Returning an empty DataFrame.")
        return pd.DataFrame()

    # Get unique values from the specified column
    unique_values = df[column_name].unique()
    # print(f"Unique values in column '{column_name}': {unique_values}") #Debugging

    reordered_rows = []
    exhausted_values = set()
    all_exhausted = False

    # Create a dictionary to keep track of the starting index for each unique value.
    start_indices = {value: 0 for value in unique_values}

    while not all_exhausted:
        all_exhausted = True # Assume all are exhausted, until proven otherwise
        for value in unique_values:
            # print(f"Processing value: {value}") # Debugging
            start_index = start_indices[value]
            # print(f"Starting index for value '{value}': {start_index}") # Debugging

            # Extract up to value_to_extract rows for the current unique value
            rows_for_value = df[df[column_name] == value].iloc[start_index : start_index + value_to_extract]
            # print(f"Number of rows extracted for value '{value}': {len(rows_for_value)}") # Debugging

            if not rows_for_value.empty:
                reordered_rows.extend(rows_for_value.to_dict('records'))
                start_indices[value] = start_index + len(rows_for_value)
                all_exhausted = False # Found some non-exhausted values
                # print(f"Next start index for value '{value}': {start_indices[value]}") # Debugging

        # Check for global exhaustion after the inner loop
        all_exhausted = all(start_indices[v] >= len(df[df[column_name] == v]) for v in unique_values)
        # print(f"All exhausted: {all_exhausted}") # Debugging

    # Create the new DataFrame
    reordered_df = pd.DataFrame(reordered_rows)
    return reordered_df

df_ordered = reorder_dataframe(df, "domain_hash", 3)

In [13]:
df_ordered

,domain,difficulty,problem,solution,answer,source,label,domain_hash
0,[Mathematics -> Algebra -> Other],8.0,Let $ n(\ge2) $ be a positive integer. Find th...,\nLet \( n (\geq 2) \) be a positive integer. ...,1 + \left\lceil \frac{n}{2} \right\rceil,china_team_selection_test,,Mathematics -> Algebra -> Other
1,[Mathematics -> Algebra -> Other],4.5,In the alphametic $W E \times E Y E=S C E N E$...,Problems 31-33 go together. See below.,5,HMMT_2,,Mathematics -> Algebra -> Other
2,[Mathematics -> Algebra -> Other],5.0,Let $Z$ be as in problem 15. Let $X$ be the gr...,Problems 13-15 go together. See below.,2,HMMT_2,,Mathematics -> Algebra -> Other
3,[Mathematics -> Geometry -> Plane Geometry -> ...,7.0,"In an acute scalene triangle $ABC$, points $D,...","\nIn an acute scalene triangle \(ABC\), points...",1,usa_team_selection_test,,Mathematics -> Geometry -> Plane Geometry -> T...
4,[Mathematics -> Geometry -> Plane Geometry -> ...,8.0,"Points $A$, $V_1$, $V_2$, $B$, $U_2$, $U_1$ li...","\nGiven the points \( A, V_1, V_2, B, U_2, U_1...",K \text{ is the intersection of } AB' \text{ a...,usa_team_selection_test_for_imo,,Mathematics -> Geometry -> Plane Geometry -> T...
...,...,...,...,...,...,...,...,...
4423,[Mathematics -> Discrete Mathematics -> Combin...,6.5,Fibonacci numbers are defined as follows: $F_0...,\nTo express \( a_n \) in terms of Fibonacci n...,a_n = \n\begin{cases} \nF_{k+1}^2 & \text{if }...,imo_longlists,,Mathematics -> Discrete Mathematics -> Combina...
4424,[Mathematics -> Discrete Mathematics -> Combin...,6.0,Every positive integer greater than $1000$ is ...,Consider the given problem in which every posi...,\text{No},problems_from_the_kvant_magazine,,Mathematics -> Discrete Mathematics -> Combina...
4425,[Mathematics -> Discrete Mathematics -> Combin...,7.0,Is it possible to choose $1983$ distinct posit...,\nTo determine whether it is possible to choos...,\text{yes},imo,,Mathematics -> Discrete Mathematics -> Combina...
4426,[Mathematics -> Discrete Mathematics -> Combin...,6.5,"The audience chooses two of twenty-nine cards,...",\nThe problem involves a magician and their as...,$如果两张牌不连续（29 和 1 是连续的），则选择每张牌后面的牌；如果两张牌连续，则选择它...,ToT,,Mathematics -> Discrete Mathematics -> Combina...


In [15]:
df_ordered = df_ordered.drop(["domain_hash"], axis=1)

In [16]:
df_ordered

,domain,difficulty,problem,solution,answer,source,label
0,[Mathematics -> Algebra -> Other],8.0,Let $ n(\ge2) $ be a positive integer. Find th...,\nLet \( n (\geq 2) \) be a positive integer. ...,1 + \left\lceil \frac{n}{2} \right\rceil,china_team_selection_test,
1,[Mathematics -> Algebra -> Other],4.5,In the alphametic $W E \times E Y E=S C E N E$...,Problems 31-33 go together. See below.,5,HMMT_2,
2,[Mathematics -> Algebra -> Other],5.0,Let $Z$ be as in problem 15. Let $X$ be the gr...,Problems 13-15 go together. See below.,2,HMMT_2,
3,[Mathematics -> Geometry -> Plane Geometry -> ...,7.0,"In an acute scalene triangle $ABC$, points $D,...","\nIn an acute scalene triangle \(ABC\), points...",1,usa_team_selection_test,
4,[Mathematics -> Geometry -> Plane Geometry -> ...,8.0,"Points $A$, $V_1$, $V_2$, $B$, $U_2$, $U_1$ li...","\nGiven the points \( A, V_1, V_2, B, U_2, U_1...",K \text{ is the intersection of } AB' \text{ a...,usa_team_selection_test_for_imo,
...,...,...,...,...,...,...,...
4423,[Mathematics -> Discrete Mathematics -> Combin...,6.5,Fibonacci numbers are defined as follows: $F_0...,\nTo express \( a_n \) in terms of Fibonacci n...,a_n = \n\begin{cases} \nF_{k+1}^2 & \text{if }...,imo_longlists,
4424,[Mathematics -> Discrete Mathematics -> Combin...,6.0,Every positive integer greater than $1000$ is ...,Consider the given problem in which every posi...,\text{No},problems_from_the_kvant_magazine,
4425,[Mathematics -> Discrete Mathematics -> Combin...,7.0,Is it possible to choose $1983$ distinct posit...,\nTo determine whether it is possible to choos...,\text{yes},imo,
4426,[Mathematics -> Discrete Mathematics -> Combin...,6.5,"The audience chooses two of twenty-nine cards,...",\nThe problem involves a magician and their as...,$如果两张牌不连续（29 和 1 是连续的），则选择每张牌后面的牌；如果两张牌连续，则选择它...,ToT,


In [17]:
df_ordered = df_ordered.reset_index()

In [18]:
with open("omni-math-doccano.jsonl", "w") as f:
    for row in df_ordered.iterrows():
        json.dump(dict(row[1]), f)
        f.write('\n')